# IEOR 142 - Final Project - Predicting the 2022 World Cup

In [1]:
#Loading Dependencies

import pandas as pd
import numpy as np
import datetime
import random
from itertools import combinations
from collections import Counter

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [2]:
"""Functions"""

def time_since(i):
    c = i.split(' -')[0]
    format = '%b %d %Y'
    a = datetime.datetime.strptime(c, format)
    b = datetime.datetime.today() - a
    return b.days

def winner(home_score, away_score):
    if home_score>away_score:
        return 1
    if home_score<away_score:
        return 0
    if home_score==away_score:
        return 0.5

def winner_binary(home_score, away_score):
    if home_score>away_score:
        return 1
    return 0

#Extracting year from date_time:
def year_extract(date_time):
    format = '%Y-%m-%d'  # The format
    datetime_str = datetime.datetime.strptime(date_time, format)
    return datetime_str.year

# Function to convert string to datetime:
def convert(date_time):
    format = '%Y-%m-%d'  # The format
    datetime_str = datetime.datetime.strptime(date_time, format)
    return datetime_str

#Convert model probabilities to predictions:
def convert_to_result(prediction):
    '''Input : nested 3D array
        Output : array of (-1,0,1)'''
    
    result = []
    for i in prediction:
        if (i[0] > i[1] and i[0] > i[2]):
            result.append(-1)
        elif (i[1] > i[0] and i[1] > i[2]):
            result.append(0)
        elif (i[2] > i[0] and i[2] > i[1]):
            result.append(1)
        else:
            result.append('Edge Case')
            
    return result

In [3]:
#Kaggle Data

#FIFA rankings for all countries dating back to the start of FIFA rankings:
fifa_ranking = pd.read_csv('fifa_ranking-2022-10-06 (1).csv') 

#Positional rankings (ex: GK_rating) and team rankings for a country's games in match-to-match format
#from 1993 to 2022:
international_matches = pd.read_csv('international_matches.csv')

#Positional rankings (ex: GK_rating) and team rank for a country's most recent game:
last_team_scores = pd.read_csv('last_team_scores.csv')

#Dataframe containing all group stage matchups in the 2022 World Cup:
qatar_group_stage = pd.read_csv('Qatar_group_stage.csv')

In [4]:
fifa_ranking

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.00,0.00,0,UEFA,1992-12-31
1,96,Syria,SYR,11.00,0.00,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.00,0.00,0,CAF,1992-12-31
3,99,Latvia,LVA,10.00,0.00,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.00,0.00,0,CAF,1992-12-31
...,...,...,...,...,...,...,...,...
63911,74,El Salvador,SLV,1330.51,1333.48,3,CONCACAF,2022-10-06
63912,75,Oman,OMA,1320.29,1323.03,0,AFC,2022-10-06
63913,76,Israel,ISR,1316.55,1316.35,0,UEFA,2022-10-06
63914,78,Georgia,GEO,1307.34,1296.46,-4,UEFA,2022-10-06


In [5]:
international_matches

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23916,2022-06-14,Moldova,Andorra,Europe,Europe,180,153,932,1040,2,...,No,Win,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23917,2022-06-14,Liechtenstein,Latvia,Europe,Europe,192,135,895,1105,0,...,No,Lose,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN
23918,2022-06-14,Chile,Ghana,South America,Africa,28,60,1526,1387,0,...,Yes,Lose,79.0,74.0,75.5,76.7,78.2,75.5,76.0,78.2
23919,2022-06-14,Japan,Tunisia,Asia,Africa,23,35,1553,1499,0,...,No,Lose,73.0,NaN,75.2,75.0,77.5,70.8,72.3,74.0


In [6]:
last_team_scores

,team,date,rank,goalkeeper_score,defense_score,offense_score,midfield_score
0,Argentina,2022-06-05,4,84.0,82.0,89.0,84.0
1,Australia,2022-06-13,42,77.0,72.0,72.0,74.0
2,Belgium,2022-06-14,2,89.0,81.0,86.0,86.0
3,Brazil,2022-06-06,1,89.0,87.0,87.0,86.0
4,Cameroon,2022-06-09,37,67.0,77.0,78.0,75.0
5,Canada,2022-06-13,38,76.0,69.0,73.0,78.0
6,Costa Rica,2022-06-14,31,88.0,72.0,70.0,69.0
7,Croatia,2022-06-13,16,82.0,78.0,77.0,84.0
8,Denmark,2022-06-13,11,85.0,80.0,78.0,80.0
9,Ecuador,2022-06-11,46,71.0,74.0,76.0,74.0


In [7]:
qatar_group_stage

,country1,country2,group
0,Qatar,Ecuador,a
1,Senegal,Netherlands,a
2,England,IR Iran,b
3,USA,Wales,b
4,France,Australia,d
...,...,...,...
59,v1,v2,x2
60,w1,w2,y1
61,x1,x2,y2
62,y1,y2,f


In [8]:
#Match-to-Match Data for international friendlies, world cup qualifiers, and local cups from 2018-2022

asia_cup = pd.read_csv('international-asian-cup-qualification-matches-2023-to-2023-stats.csv')
gold_cup = pd.read_csv('international-concacaf-gold-cup-matches-2021-to-2021-stats.csv')
copa = pd.read_csv('international-copa-america-matches-2021-to-2021-stats.csv')
arab_cup = pd.read_csv('international-fifa-arab-cup-matches-2021-to-2021-stats.csv')
wc_2018 = pd.read_csv('international-fifa-world-cup-2018-russia-matches-2018-to-2018-stats.csv')

friendly_2018 = pd.read_csv('international-international-friendlies-matches-2018-to-2018-stats.csv')
friendly_2019 = pd.read_csv('international-international-friendlies-matches-2019-to-2019-stats.csv')
friendly_2020 = pd.read_csv('international-international-friendlies-matches-2020-to-2020-stats.csv')
friendly_2021 = pd.read_csv('international-international-friendlies-matches-2021-to-2021-stats.csv')
friendly_2022 = pd.read_csv('international-international-friendlies-matches-2022-to-2022-stats (1).csv')

euro_2016 = pd.read_csv('international-uefa-euro-qualifiers-matches-2016-to-2016-stats.csv')
euro_2019_2020 = pd.read_csv('international-uefa-euro-qualifiers-matches-2019-to-2020-stats.csv')
nations_2018_2019 = pd.read_csv('international-uefa-nations-league-matches-2018-to-2019-stats.csv')
nations_2020_2021 = pd.read_csv('international-uefa-nations-league-matches-2020-to-2021-stats.csv')
nations_2022_2023 = pd.read_csv('international-uefa-nations-league-matches-2022-to-2023-stats.csv')

wc_qual_africa = pd.read_csv('international-wc-qualification-africa-matches-2022-to-2022-stats.csv')
wc_qual_asia = pd.read_csv('international-wc-qualification-asia-matches-2022-to-2022-stats.csv')
wc_qual_c = pd.read_csv('international-wc-qualification-concacaf-matches-2022-to-2022-stats.csv')
wc_qual_euro = pd.read_csv('international-wc-qualification-europe-matches-2022-to-2022-stats.csv')
wc_qual_inter = pd.read_csv('international-wc-qualification-intercontinental-playoffs-matches-2022-to-2022-stats.csv')
wc_qual_sa = pd.read_csv('international-wc-qualification-south-america-matches-2020-to-2022-stats.csv')

Here is one example of match-to-match data. All of the columns are the same for the dataframes above:

In [9]:
wc_qual_euro

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1616605200,Mar 24 2021 - 5:00pm,complete,NaN,Turkey,Netherlands,NaN,1.0,0.00,0.00,...,4.79,3.85,1.75,1.29,1.83,2.85,6.50,1.95,1.80,Atatürk Olimpiyat Stadı
1,1616615100,Mar 24 2021 - 7:45pm,complete,NaN,Portugal,Azerbaijan,NaN,1.0,0.00,0.00,...,1.03,14.00,34.00,1.07,1.23,1.60,2.20,4.00,1.22,Allianz Stadium
2,1616615100,Mar 24 2021 - 7:45pm,complete,NaN,Serbia,Republic of Ireland,NaN,1.0,0.00,0.00,...,1.65,3.60,6.40,1.45,2.36,4.33,10.00,2.20,1.62,Stadion Rajko Mitić
3,1616615100,Mar 24 2021 - 7:45pm,complete,NaN,Finland,Bosnia-Herzegovina,NaN,1.0,0.00,0.00,...,2.75,3.10,2.83,1.50,2.51,4.50,11.00,2.00,1.75,Helsingin olympiastadion
4,1616615100,Mar 24 2021 - 7:45pm,complete,NaN,France,Ukraine,NaN,1.0,0.00,0.00,...,1.36,4.40,9.00,1.25,1.80,3.20,5.50,2.25,1.57,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,1648151100,Mar 24 2022 - 7:45pm,complete,NaN,Portugal,Turkey,NaN,NaN,2.13,2.10,...,1.12,5.00,6.75,1.24,1.75,2.95,5.50,2.00,1.75,Estádio Do Dragão
255,1648579500,Mar 29 2022 - 6:45pm,complete,NaN,Poland,Sweden,NaN,NaN,2.00,2.00,...,1.66,2.90,2.25,1.50,2.60,4.60,9.75,2.10,1.67,NaN
256,1648579500,Mar 29 2022 - 6:45pm,complete,NaN,Portugal,FYR Macedonia,NaN,NaN,2.22,1.91,...,1.03,6.50,10.00,1.22,1.70,2.65,4.60,2.50,1.50,Estádio Do Dragão
257,1654109100,Jun 01 2022 - 6:45pm,complete,NaN,Scotland,Ukraine,NaN,NaN,2.30,1.50,...,1.60,3.00,2.25,1.53,2.60,4.50,9.00,2.10,1.67,Hampden Park


In [10]:
#Aggregated data by country for international friendlies, world cup qualifiers, and local cups
#from 2018-2022

asia_cup_team = pd.read_csv('international-asian-cup-qualification-teams-2023-to-2023-stats.csv')
gold_cup_team = pd.read_csv('international-concacaf-gold-cup-teams-2021-to-2021-stats.csv')
copa_team = pd.read_csv('international-copa-america-teams-2021-to-2021-stats.csv')
arab_cup_team = pd.read_csv('international-fifa-arab-cup-teams-2021-to-2021-stats.csv')
wc_2018_team = pd.read_csv('international-fifa-world-cup-2018-russia-teams-2018-to-2018-stats.csv')

friendly_2018_team = pd.read_csv('international-international-friendlies-teams-2018-to-2018-stats.csv')
friendly_2019_team = pd.read_csv('international-international-friendlies-teams-2019-to-2019-stats.csv')
friendly_2020_team = pd.read_csv('international-international-friendlies-teams-2020-to-2020-stats.csv')
friendly_2021_team = pd.read_csv('international-international-friendlies-teams-2021-to-2021-stats.csv')
friendly_2022_team = pd.read_csv('international-international-friendlies-teams-2022-to-2022-stats.csv')

euro_2019_2020_team = pd.read_csv('international-uefa-euro-qualifiers-teams-2019-to-2020-stats.csv')
nations_2018_2019_team = pd.read_csv('international-uefa-nations-league-teams-2018-to-2019-stats.csv')
nations_2020_2021_team = pd.read_csv('international-uefa-nations-league-teams-2020-to-2021-stats.csv')
nations_2022_2023_team = pd.read_csv('international-uefa-nations-league-teams-2022-to-2023-stats.csv')

wc_qual_africa_team = pd.read_csv('international-wc-qualification-africa-teams-2022-to-2022-stats.csv')
wc_qual_asia_team = pd.read_csv('international-wc-qualification-asia-teams-2022-to-2022-stats.csv')
wc_qual_c_team = pd.read_csv('international-wc-qualification-concacaf-teams-2022-to-2022-stats.csv')
wc_qual_euro_team = pd.read_csv('international-wc-qualification-europe-teams-2022-to-2022-stats.csv')
wc_qual_inter_team = pd.read_csv('international-wc-qualification-intercontinental-playoffs-teams-2022-to-2022-stats.csv')
wc_qual_sa_team = pd.read_csv('international-wc-qualification-south-america-teams-2020-to-2022-stats.csv')

joined = pd.read_csv('DataScience\joined_data.csv')

#Dataframe of teams at Qatar 2022 World Cup:
qatar_teams = pd.read_csv('Qatar2022-teams.csv')

Here is one example of aggregated team data. All of the columns are the same for the dataframes above:

In [11]:
wc_qual_euro_team

,team_name,common_name,season,country,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,...,goals_conceded_min_61_to_70,goals_conceded_min_71_to_80,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage
0,Finland National Team,Finland,2022,Finland,8,4,4,0,3,1,...,2,2,1,25,25,25,38,50,25,0
1,Iceland National Team,Iceland,2022,Iceland,10,5,5,0,2,1,...,1,2,3,30,40,20,50,40,60,10
2,Gibraltar National Team,Gibraltar,2022,Gibraltar,10,5,5,0,0,0,...,6,3,9,0,0,0,100,100,100,20
3,Belarus National Team,Belarus,2022,Belarus,8,4,4,0,1,1,...,2,2,3,0,0,0,88,75,100,13
4,Czech Republic National Team,Czech Republic,2022,Czech Republic,9,4,5,0,4,2,...,2,0,2,22,50,0,33,0,60,0
5,Montenegro National Team,Montenegro,2022,Montenegro,10,5,5,0,3,1,...,2,1,1,30,40,20,40,40,40,0
6,Switzerland National Team,Switzerland,2022,Switzerland,8,4,4,0,5,3,...,0,0,0,38,25,50,0,0,0,25
7,Serbia National Team,Serbia,2022,Serbia,8,4,4,0,6,3,...,0,1,2,25,25,25,0,0,0,13
8,Andorra National Team,Andorra,2022,Andorra,10,5,5,0,2,1,...,0,5,4,0,0,0,80,80,80,10
9,Austria National Team,Austria,2022,Austria,11,5,6,0,5,3,...,2,2,2,9,0,17,45,40,50,0


## Data Cleaning:

Here we are replacing inconsistencies among the fifa_ranking dataframe and renaming countries to match the names of those countries in other dataframes. We also take the most latest rankings for each country in the year from 2018 to 2022 below:

In [12]:
fifa_ranking = fifa_ranking.replace(['IR Iran'], 'Iran')
fifa_ranking = fifa_ranking.replace(['USA'], 'USMNT')
fifa_ranking = fifa_ranking.replace(['United Arab Emirates'], 'UAE')
fifa_ranking = fifa_ranking.replace(['Chinese Taipei'], 'Taiwan')
fifa_ranking = fifa_ranking.replace(['Kyrgyz Republic'], 'Kyrgyzstan')
fifa_ranking = fifa_ranking.replace(['Morroco A'], 'Morroco')
fifa_ranking = fifa_ranking.replace(['Türkiye'], 'Turkey')
fifa_ranking = fifa_ranking.replace(['North Macedonia'], 'FYR Macedonia')
fifa_ranking = fifa_ranking.replace(['Korea Republic'], 'South Korea')
fifa_ranking = fifa_ranking.replace(['China PR'], 'China') 
fifa_ranking = fifa_ranking.replace(['Korea DPR'], 'North Korea')


fifa_ranking['year'] = fifa_ranking['rank_date'].astype(str).apply(year_extract)
fifa_ranking['year'] = fifa_ranking['year'].astype(float).dropna().astype(int)
fifa_ranking_2022 = fifa_ranking[fifa_ranking['rank_date']=='2022-10-06']
fifa_ranking_2022 = fifa_ranking_2022.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

In [13]:
fifa_ranking_2021 = fifa_ranking[fifa_ranking['rank_date']=='2021-12-23']
fifa_ranking_2021 = fifa_ranking_2021.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

fifa_ranking_2020 = fifa_ranking[fifa_ranking['rank_date']=='2020-12-10']
fifa_ranking_2020 = fifa_ranking_2020.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

fifa_ranking_2019 = fifa_ranking[fifa_ranking['rank_date']=='2019-12-19']
fifa_ranking_2019 = fifa_ranking_2019.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

fifa_ranking_2018 = fifa_ranking[fifa_ranking['rank_date']=='2018-12-20']
fifa_ranking_2018 = fifa_ranking_2018.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

In [14]:
fifa_rank_2018_to_2022 = pd.concat([fifa_ranking_2018, fifa_ranking_2019, 
fifa_ranking_2020, fifa_ranking_2021, fifa_ranking_2022])
fifa_rank_2018_to_2022

,rank,country_full,country_abrv,total_points,previous_points,rank_change,year
57608,1,Belgium,BEL,1727.00,1727.00,0,2018
57793,2,France,FRA,1726.00,1726.00,0,2018
57713,3,Brazil,BRA,1676.00,1676.00,0,2018
57607,4,Croatia,CRO,1634.00,1634.00,0,2018
57684,5,England,ENG,1631.00,1631.00,0,2018
...,...,...,...,...,...,...,...
63776,207,Sri Lanka,SRI,825.25,825.25,0,2022
63777,208,US Virgin Islands,VIR,823.97,823.97,0,2022
63778,209,British Virgin Islands,VGB,809.32,809.32,0,2022
63837,210,Anguilla,AIA,790.74,790.74,0,2022


Replacing name inconsistencies again in this last_team_scores dataframe and displaying teams that have NAN values in the international_matches dataframe. Under the imputing values section, we will impute the positional score values shown in the impute_data dataframe below into all instances of these positional scores showing up in our match-to-match data in the international_matches dataframe.

In [15]:
last_team_scores = last_team_scores.replace(['USA'], 'USMNT')
last_team_scores = last_team_scores.replace(['Korea Republic'], 'South Korea')
last_team_scores = last_team_scores.replace(['IR Iran'], 'Iran')
impute_data = last_team_scores[(last_team_scores['team'] == 'USMNT') | (last_team_scores['team'] == 'Iran') | 
(last_team_scores['team'] == 'South Korea') | (last_team_scores['team'] == 'Qatar')]
impute_data

,team,date,rank,goalkeeper_score,defense_score,offense_score,midfield_score
14,Iran,2022-06-12,21,73.0,69.0,75.0,69.0
16,South Korea,2022-06-14,29,75.0,73.0,80.0,74.0
22,Qatar,2022-03-29,52,50.0,78.0,80.0,79.0
29,USMNT,2022-06-14,15,77.0,76.0,78.0,76.0


## Imputing Values:

Replacing more name inconsistencies:

In [16]:
international_matches = international_matches.replace(['IR Iran'], 'Iran')
international_matches = international_matches.replace(['USA'], 'USMNT')
international_matches = international_matches.replace(['United Arab Emirates'], 'UAE')
international_matches = international_matches.replace(['Chinese Taipei'], 'Taiwan')
international_matches = international_matches.replace(['Kyrgyz Republic'], 'Kyrgyzstan')
international_matches = international_matches.replace(['Morroco A'], 'Morroco')
international_matches = international_matches.replace(['Türkiye'], 'Turkey')
international_matches = international_matches.replace(['North Macedonia'], 'FYR Macedonia')
international_matches = international_matches.replace(['Korea Republic'], 'South Korea')
international_matches = international_matches.replace(['China PR'], 'China') 
international_matches = international_matches.replace(['Korea DPR'], 'North Korea')

Only selecting matches from 2018 and on to give us the most accurate representation of the current team's form while providing the most data possible:

In [17]:
year = list(international_matches['date'].str.split('-', expand = True)[0])
international_matches['year'] = year
international_matches['year'] = international_matches['year'].apply(int)
international_matches = international_matches[international_matches['year'] >= 2018]
qt_home = international_matches[international_matches['home_team']=='Qatar']
qt_away = international_matches[international_matches['away_team']=='Qatar']
us_home = international_matches[international_matches['home_team']=='USMNT']
us_away = international_matches[international_matches['away_team']=='USMNT']
ir_home = international_matches[international_matches['home_team']=='Iran']
ir_away = international_matches[international_matches['away_team']=='Iran']
sk_home = international_matches[international_matches['home_team']=='South Korea']
sk_away = international_matches[international_matches['away_team']=='South Korea']
qt_away

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,year
20121,2018-03-21,Iraq,Qatar,Asia,Asia,83,101,405,332,2,...,Lose,NaN,NaN,NaN,NaN,69.5,NaN,NaN,NaN,2018
20713,2018-10-16,Uzbekistan,Qatar,Asia,Asia,96,94,1245,1247,2,...,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
20740,2018-11-14,Switzerland,Qatar,Europe,Asia,8,96,1598,1248,0,...,Lose,82.0,NaN,78.8,74.3,79.0,NaN,NaN,NaN,2018
20831,2018-11-19,Iceland,Qatar,Europe,Asia,36,96,1458,1248,2,...,Draw,69.0,NaN,72.8,72.7,76.8,NaN,NaN,NaN,2018
20925,2019-01-13,North Korea,Qatar,Asia,Asia,109,93,1196,1258,0,...,Lose,NaN,NaN,NaN,68.0,NaN,NaN,NaN,NaN,2019
20938,2019-01-17,Saudi Arabia,Qatar,Asia,Asia,69,93,1335,1258,0,...,Lose,70.0,NaN,72.0,68.7,73.0,NaN,NaN,NaN,2019
20951,2019-01-25,South Korea,Qatar,Asia,Asia,53,93,1405,1258,0,...,Lose,74.0,NaN,71.8,74.3,77.8,NaN,NaN,NaN,2019
20955,2019-01-29,UAE,Qatar,Asia,Asia,79,93,1309,1258,0,...,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
20956,2019-02-01,Japan,Qatar,Asia,Asia,50,93,1414,1258,1,...,Lose,72.0,NaN,76.0,76.3,79.5,NaN,NaN,NaN,2019
21114,2019-06-05,Brazil,Qatar,South America,Asia,3,55,1676,1398,2,...,Win,86.0,NaN,87.0,88.7,86.2,NaN,NaN,NaN,2019


Here we rename the columns to replace inconsistencies between dataframes:

In [18]:
# Qatar Home Games:
qatar_home = impute_data[impute_data['team']=='Qatar'].rename(columns={'goalkeeper_score' : 'home_team_goalkeeper_score',
                                                  'defense_score' : 'home_team_defense_score',
                                                  'offense_score' : 'home_team_offense_score',
                                                  'midfield_score' : 'home_team_midfield_score'})

In [19]:
# Qatar Away Games:
qatar_away = impute_data[impute_data['team']=='Qatar'].rename(columns={'goalkeeper_score' : 'away_team_goalkeeper_score',
                                                  'defense_score' : 'away_team_defense_score',
                                                  'offense_score' : 'away_team_offense_score',
                                                  'midfield_score' : 'away_team_midfield_score'})

In [20]:
# USMNT Home Games:
USMNT_home = impute_data[impute_data['team']=='USMNT'].rename(columns={'goalkeeper_score' : 'home_team_goalkeeper_score',
                                                  'defense_score' : 'home_team_defense_score',
                                                  'offense_score' : 'home_team_offense_score',
                                                  'midfield_score' : 'home_team_midfield_score'})

In [21]:
# USMNT Away Games:
USMNT_away = impute_data[impute_data['team']=='USMNT'].rename(columns={'goalkeeper_score' : 'awayaway_team_goalkeeper_score',
                                                  'defense_score' : 'away_team_defense_score',
                                                  'offense_score' : 'away_team_offense_score',
                                                  'midfield_score' : 'away_team_midfield_score'})

In [22]:
# Iran Home Games:
iran_home = impute_data[impute_data['team']=='Iran'].rename(columns={'goalkeeper_score' : 'home_team_goalkeeper_score',
                                                  'defense_score' : 'home_team_defense_score',
                                                  'offense_score' : 'home_team_offense_score',
                                                  'midfield_score' : 'home_team_midfield_score'})

In [23]:
# Iran Away Games:
iran_away = impute_data[impute_data['team']=='Iran'].rename(columns={'goalkeeper_score' : 'awayaway_team_goalkeeper_score',
                                                  'defense_score' : 'away_team_defense_score',
                                                  'offense_score' : 'away_team_offense_score',
                                                  'midfield_score' : 'away_team_midfield_score'})

In [24]:
# South Korea Home Games:
korea_home = impute_data[impute_data['team']=='South Korea'].rename(columns={'goalkeeper_score' : 'home_team_goalkeeper_score',
                                                  'defense_score' : 'home_team_defense_score',
                                                  'offense_score' : 'home_team_offense_score',
                                                  'midfield_score' : 'home_team_midfield_score'})

In [25]:
# South Korea Away Games:
korea_away = impute_data[impute_data['team']=='South Korea'].rename(columns={'goalkeeper_score' : 'away_team_goalkeeper_score',
                                                  'defense_score' : 'away_team_defense_score',
                                                  'offense_score' : 'away_team_offense_score',
                                                  'midfield_score': 'home_team_midfield_score'})

Here we impute the country's scores for both home and away games. Any commented out lines of code indicate that there are no NAN values for that country's positional score, indicating no need to impute any data for that column.

In [26]:
qt_home.loc[:,'home_team_goalkeeper_score'] = qt_home['home_team_goalkeeper_score'].replace(
    np.nan, list(qatar_home['home_team_goalkeeper_score'])[0])

qt_home.loc[:,'home_team_mean_defense_score'] = qt_home['home_team_mean_defense_score'].replace(
    np.nan, list(qatar_home['home_team_defense_score'])[0])

qt_home.loc[:,'home_team_mean_offense_score'] = qt_home['home_team_mean_offense_score'].replace(
    np.nan, list(qatar_home['home_team_offense_score'])[0])

qt_home.loc[:,'home_team_mean_midfield_score'] = qt_home['home_team_mean_midfield_score'].replace(
    np.nan, list(qatar_home['home_team_midfield_score'])[0])

In [27]:
qt_away.loc[:,'away_team_goalkeeper_score'] = qt_away['away_team_goalkeeper_score'].replace(
    np.nan, list(qatar_away['away_team_goalkeeper_score'])[0])

qt_away.loc[:,'away_team_mean_defense_score'] = qt_away['away_team_mean_defense_score'].replace(
    np.nan, list(qatar_away['away_team_defense_score'])[0])

qt_away.loc[:,'away_team_mean_offense_score'] = qt_away['away_team_mean_offense_score'].replace(
    np.nan, list(qatar_away['away_team_offense_score'])[0])

qt_away.loc[:,'away_team_mean_midfield_score'] = qt_away['away_team_mean_midfield_score'].replace(
    np.nan, list(qatar_away['away_team_midfield_score'])[0])

In [28]:
us_home.loc[:,'home_team_goalkeeper_score'] = us_home['home_team_goalkeeper_score'].replace(
    np.nan, list(USMNT_home['home_team_goalkeeper_score'])[0])

us_home.loc[:,'home_team_mean_defense_score'] = us_home['home_team_mean_defense_score'].replace(
    np.nan, list(USMNT_home['home_team_defense_score'])[0])

us_home.loc[:,'home_team_mean_offense_score'] = us_home['home_team_mean_offense_score'].replace(
    np.nan, list(USMNT_home['home_team_offense_score'])[0])

us_home.loc[:,'home_team_mean_midfield_score'] = us_home['home_team_mean_midfield_score'].replace(
    np.nan, list(USMNT_home['home_team_midfield_score'])[0])

In [29]:
# us_away.loc[:,'away_team_goalkeeper_score'] = us_away['away_team_goalkeeper_score'].replace(
#     np.nan, list(USMNT_away['away_team_goalkeeper_score'])[0])

us_away.loc[:,'away_team_mean_defense_score'] = us_away['away_team_mean_defense_score'].replace(
    np.nan, list(USMNT_away['away_team_defense_score'])[0])

us_away.loc[:,'away_team_mean_offense_score'] = us_away['away_team_mean_offense_score'].replace(
    np.nan, list(USMNT_away['away_team_offense_score'])[0])

us_away.loc[:,'away_team_mean_midfield_score'] = us_away['away_team_mean_midfield_score'].replace(
    np.nan, list(USMNT_away['away_team_midfield_score'])[0])

In [30]:
ir_home.loc[:,'home_team_goalkeeper_score'] = ir_home['home_team_goalkeeper_score'].replace(
    np.nan, list(iran_home['home_team_goalkeeper_score'])[0])

ir_home.loc[:,'home_team_mean_defense_score'] = ir_home['home_team_mean_defense_score'].replace(
    np.nan, list(iran_home['home_team_defense_score'])[0])

ir_home.loc[:,'home_team_mean_offense_score'] = ir_home['home_team_mean_offense_score'].replace(
    np.nan, list(iran_home['home_team_offense_score'])[0])

ir_home.loc[:,'home_team_mean_midfield_score'] =ir_home['home_team_mean_midfield_score'].replace(
    np.nan, list(iran_home['home_team_midfield_score'])[0])

In [31]:
# ir_away.loc[:,'away_team_goalkeeper_score'] = ir_away['away_team_goalkeeper_score'].replace(
#     np.nan, list(iran_away['away_team_goalkeeper_score'])[0])

ir_away.loc[:,'away_team_mean_defense_score'] = ir_away['away_team_mean_defense_score'].replace(
    np.nan, list(iran_away['away_team_defense_score'])[0])

ir_away.loc[:,'away_team_mean_offense_score'] = ir_away['away_team_mean_offense_score'].replace(
    np.nan, list(iran_away['away_team_offense_score'])[0])

ir_away.loc[:,'away_team_mean_midfield_score'] = ir_away['away_team_mean_midfield_score'].replace(
    np.nan, list(iran_away['away_team_midfield_score'])[0])

In [32]:
sk_home.loc[:,'home_team_goalkeeper_score'] = sk_home['home_team_goalkeeper_score'].replace(
    np.nan, list(korea_home['home_team_goalkeeper_score'])[0])

sk_home.loc[:,'home_team_mean_defense_score'] = sk_home['home_team_mean_defense_score'].replace(
    np.nan, list(korea_home['home_team_defense_score'])[0])

sk_home.loc[:,'home_team_mean_offense_score'] = sk_home['home_team_mean_offense_score'].replace(
    np.nan, list(korea_home['home_team_offense_score'])[0])

sk_home.loc[:,'home_team_mean_midfield_score'] =sk_home['home_team_mean_midfield_score'].replace(
    np.nan, list(korea_home['home_team_midfield_score'])[0])

In [33]:
sk_away.loc[:,'away_team_goalkeeper_score'] = sk_away['away_team_goalkeeper_score'].replace(
    np.nan, list(korea_away['away_team_goalkeeper_score'])[0])

sk_away.loc[:,'away_team_mean_defense_score'] = sk_away['away_team_mean_defense_score'].replace(
    np.nan, list(korea_away['away_team_defense_score'])[0])

sk_away.loc[:,'away_team_mean_offense_score'] = sk_away['away_team_mean_offense_score'].replace(
    np.nan, list(korea_away['away_team_offense_score'])[0])

# sk_away.loc[:,'away_team_mean_midfield_score'] = sk_away['away_team_mean_midfield_score'].replace(
#     np.nan, list(korea_away['away_team_midfield_score'])[0])

Here we concatenate all of the above dataframes with the original international_matches dataframe and then drop NAN values to get total_inter. Total_inter has the same schema as international_matches, but now these four countries have ratings match-to-match.

In [34]:
total_inter = pd.concat([qt_home, qt_away, us_home, us_away, ir_home, ir_away,
                         sk_home, sk_away, international_matches])
total_inter = total_inter.dropna()
total_inter

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,year
20505,2018-09-07,Qatar,China,Asia,Asia,98,75,1236,1320,1,...,Win,50.0,72.0,78.0,80.0,79.0,70.2,72.3,71.5,2018
20632,2018-10-12,Qatar,Ecuador,Asia,South America,94,58,1247,1378,4,...,Win,50.0,74.0,78.0,80.0,79.0,74.0,75.7,75.5,2018
20889,2018-12-27,Qatar,Algeria,Asia,Africa,93,67,1258,1347,0,...,Lose,50.0,67.0,78.0,80.0,79.0,77.2,80.0,80.8,2018
20899,2018-12-31,Qatar,Iran,Asia,Asia,93,29,1258,1481,1,...,Lose,50.0,72.0,78.0,80.0,79.0,69.0,73.3,71.0,2018
21279,2019-06-23,Qatar,Argentina,Asia,South America,55,11,1396,1582,0,...,Lose,50.0,81.0,78.0,80.0,79.0,82.5,90.3,84.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,2022-06-14,Poland,Belgium,Europe,Europe,26,2,1544,1827,0,...,Lose,87.0,89.0,75.2,84.7,75.8,80.8,85.7,85.5,2022
23911,2022-06-14,Ukraine,Republic of Ireland,Europe,Europe,27,47,1535,1449,1,...,Draw,75.0,75.0,74.8,78.7,80.0,76.5,72.7,73.8,2022
23912,2022-06-14,Bosnia and Herzegovina,Finland,Europe,Europe,59,57,1388,1406,3,...,Win,76.0,83.0,74.2,77.0,78.0,70.0,72.3,73.5,2022
23913,2022-06-14,Romania,Montenegro,Europe,Europe,48,70,1446,1342,0,...,Lose,77.0,65.0,73.5,73.7,75.0,76.2,74.7,68.2,2022


# More Data Cleaning:

These dataframes all have the same schema, so we can concatenate all of these match-to-match dataframes below:

In [35]:
full_data = pd.concat([asia_cup,gold_cup,copa,arab_cup,wc_2018,
                     friendly_2018, friendly_2019, friendly_2020,
                     friendly_2021, friendly_2022, euro_2019_2020, nations_2018_2019,
                     nations_2020_2021, nations_2022_2023, wc_qual_africa, wc_qual_asia, wc_qual_c,
                     wc_qual_euro, wc_qual_inter, wc_qual_sa], axis=0).reset_index()

Splitting on our data_time column and taking the year from that split to create a new year column:

In [36]:
year = list(full_data['date_GMT'].str.split(' ', expand = True)[2])
full_data['year'] = year
full_data['year'] = full_data['year'].apply(int)
fifa_rank_2018_to_2022['year']= fifa_rank_2018_to_2022['year'].apply(int)

Joining two dataframes to team rank data to all match-to-match data when a country is the home team and also when a country is the away team:

In [37]:
full_data = full_data.merge(fifa_rank_2018_to_2022,
                            left_on=['home_team_name','year'], right_on=['country_full','year'],
                            how='left')
full_data = full_data.merge(fifa_rank_2018_to_2022,
                            left_on=['away_team_name','year'], right_on=['country_full','year'],
                            how='left',suffixes = ('_home', '_away'))

Replacing null values in 'home_team_goal_timings':

In [38]:
full_data['home_team_goal_timings'] = full_data['home_team_goal_timings'].replace(np.nan, -1)
full_data['away_team_goal_timings'] = full_data['away_team_goal_timings'].replace(np.nan, -1)
full_data[full_data['home_team_goal_timings'].isnull()]['home_team_goal_timings']

Series([], Name: home_team_goal_timings, dtype: object)

Filtering out all cancelled games, applying our time since function to create a days_since column. This gives us a number with the days it has been from the time of that game between the start of the world cup. We also rename two columns for readability and drop unneeded columns:

In [39]:
full_data['days_since'] = full_data['date_GMT'].apply(time_since)
full_data = full_data[full_data['days_since']>=0]
full_data = full_data[full_data['status']!='canceled']
full_data = full_data.drop(['Game Week','referee', 'attendance',
                            'index', 'timestamp', 'status', 'stadium_name'], axis=1)
full_data = full_data.rename(columns= {'Pre-Match PPG (Home)' : 'Pre_Match_PPG_Home',
                                       'Pre-Match PPG (Away)' : 'Pre_Match_PPG_Away'})

Checking for null values. Here we can also see all of the current features after our merges:

In [40]:
for i in full_data.columns:
    print(i, full_data[i].isna().sum())

date_GMT 0
home_team_name 0
away_team_name 0
Pre_Match_PPG_Home 0
Pre_Match_PPG_Away 0
home_ppg 0
away_ppg 0
home_team_goal_count 0
away_team_goal_count 0
total_goal_count 0
total_goals_at_half_time 0
home_team_goal_count_half_time 0
away_team_goal_count_half_time 0
home_team_goal_timings 0
away_team_goal_timings 0
home_team_corner_count 0
away_team_corner_count 0
home_team_yellow_cards 0
home_team_red_cards 0
away_team_yellow_cards 0
away_team_red_cards 0
home_team_first_half_cards 0
home_team_second_half_cards 0
away_team_first_half_cards 0
away_team_second_half_cards 0
home_team_shots 0
away_team_shots 0
home_team_shots_on_target 0
away_team_shots_on_target 0
home_team_shots_off_target 0
away_team_shots_off_target 0
home_team_fouls 0
away_team_fouls 0
home_team_possession 0
away_team_possession 0
Home Team Pre-Match xG 0
Away Team Pre-Match xG 0
team_a_xg 0
team_b_xg 0
average_goals_per_match_pre_match 0
btts_percentage_pre_match 0
over_15_percentage_pre_match 0
over_25_percentage_p

In [41]:
# #Creating target variables for group and knockout stages:
# full_data['home_team_result'] = full_data.apply(lambda x: winner(x.home_team_goal_count, x.away_team_goal_count), axis=1)
# full_data['home_team_result_binary'] = full_data.apply(lambda x: winner_binary(x.home_team_goal_count, x.away_team_goal_count), axis=1)

In [42]:
text1 = full_data.columns.values.tolist()
listToStr = ','.join([str(elem) for elem in text1])
list1 = listToStr.replace(',', ' + ')
print(list1)

date_GMT + home_team_name + away_team_name + Pre_Match_PPG_Home + Pre_Match_PPG_Away + home_ppg + away_ppg + home_team_goal_count + away_team_goal_count + total_goal_count + total_goals_at_half_time + home_team_goal_count_half_time + away_team_goal_count_half_time + home_team_goal_timings + away_team_goal_timings + home_team_corner_count + away_team_corner_count + home_team_yellow_cards + home_team_red_cards + away_team_yellow_cards + away_team_red_cards + home_team_first_half_cards + home_team_second_half_cards + away_team_first_half_cards + away_team_second_half_cards + home_team_shots + away_team_shots + home_team_shots_on_target + away_team_shots_on_target + home_team_shots_off_target + away_team_shots_off_target + home_team_fouls + away_team_fouls + home_team_possession + away_team_possession + Home Team Pre-Match xG + Away Team Pre-Match xG + team_a_xg + team_b_xg + average_goals_per_match_pre_match + btts_percentage_pre_match + over_15_percentage_pre_match + over_25_percentage_p

Due to missing values in possession columns, we will take the country's average possession and impute these averages where there are no values recorded. (Need steps we took to get final_df inputted below):

## Revist this: do we even need this if possession is not used in model?

In [43]:
# From Tyler's code. Need all other team data here to be concatenated for possession calcuation:
qualification_full = pd.concat([asia_cup_team,gold_cup_team,copa_team,arab_cup_team,wc_2018_team,
                     friendly_2018_team, friendly_2019_team, friendly_2020_team,
                     friendly_2021_team, friendly_2022_team, euro_2019_2020_team, nations_2018_2019_team,
                     nations_2020_2021_team, nations_2022_2023_team, wc_qual_africa_team, wc_qual_asia_team, 
                     wc_qual_c_team, wc_qual_euro_team, wc_qual_inter_team, wc_qual_sa_team])

In [44]:
qualification_full = qualification_full.reset_index().drop(columns = ['index'])
possession = qualification_full[['country', 'average_possession', 'average_possession_home', 'average_possession_away']]
possession = possession.groupby('country').agg(np.average).reset_index()
possession

,country,average_possession,average_possession_home,average_possession_away
0,Afghanistan,40.400000,29.000000,41.400000
1,Albania,51.444444,46.555556,37.666667
2,Algeria,53.857143,48.428571,44.142857
3,Andorra,37.111111,38.000000,31.333333
4,Angola,50.000000,26.750000,37.000000
...,...,...,...,...
203,Vietnam,36.800000,30.000000,15.400000
204,Wales,49.000000,48.600000,37.400000
205,Yemen,41.750000,24.000000,37.750000
206,Zambia,46.000000,25.500000,37.166667


This function defines our target variable 'who_won':

In [45]:
#1 means : home team scored first
#0 means : neither home team or away team scored
#-1 means : away team scored first


first_score_country = []
who_won = []

for index, row in full_data.iterrows():
    temp_home_team_name = row['home_team_name']
    temp_away_team_name = row['away_team_name']
    
    home_first_time = str(row['home_team_goal_timings']).split(',')[0]
    away_first_time = str(row['away_team_goal_timings']).split(',')[0]
    
    if (home_first_time > away_first_time):
        first_score_country.append(1)
    elif (home_first_time < away_first_time):
        first_score_country.append(-1)
    else:
        first_score_country.append(0)
        
    temp_home_team_goal_cnt = row['home_team_goal_count']
    temp_away_team_goal_cnt = row['away_team_goal_count']
    
    if (temp_home_team_goal_cnt > temp_away_team_goal_cnt):
        who_won.append(1)
    elif (temp_home_team_goal_cnt < temp_away_team_goal_cnt):
        who_won.append(-1)
    else:
        who_won.append(0)

Here we create a new column for which country scored first and for our target variable that displays a 1 if the home team wins, a 0 if there is a draw, and a -1 if the home team lost:

In [46]:
full_data['first_score_country'] = first_score_country
full_data['who_won'] = who_won

In [47]:
#full_data['home_team_result'] = full_data['home_team_result'].astype(int)
#full_data['home_team_result_binary'] = full_data['home_team_result_binary'].astype(int)

Renames home team name to team1 and away team name to team2 for consistency:

In [48]:
def converting_home_and_away(df):
    '''Input : a df
    what it does : it converts home to team1 and away to team2 on the column index'''
    
    for column in list(df.columns):
        if 'home_team' in str(column):
            df = df.rename(columns = {column : column.replace('home_team', 'team1')})
        elif 'away_team' in column:
            df = df.rename(columns = {column : column.replace('away_team', 'team2')})
        elif 'home' in column:
            df = df.rename(columns = {column : column.replace('home', 'team1')})
        elif 'away' in column:
            df = df.rename(columns = {column : column.replace('away', 'team2')})
        else:
            continue
            
    return df

In [49]:
full_data = converting_home_and_away(full_data)

In [50]:
full_data

,date_GMT,team1_name,team2_name,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,team1_goal_count,team2_goal_count,total_goal_count,...,rank_change_team1,rank_team2,country_full_team2,country_abrv_team2,total_points_team2,previous_points_team2,rank_change_team2,days_since,first_score_country,who_won
0,Oct 07 2021 - 12:00pm,Indonesia,Taiwan,0.00,0.00,2.40,0.00,2,1,3,...,-2.0,158.0,Taiwan,TPE,1017.78,1017.78,0.0,435,-1,1
1,Oct 09 2021 - 5:30pm,Guam,Cambodia,0.00,0.00,0.00,1.40,0,1,1,...,0.0,171.0,Cambodia,CAM,966.61,970.42,1.0,433,-1,-1
2,Oct 11 2021 - 1:00pm,Taiwan,Indonesia,0.00,3.00,0.00,2.40,0,3,3,...,0.0,164.0,Indonesia,IDN,992.31,981.18,-2.0,431,-1,-1
3,Oct 12 2021 - 4:00pm,Cambodia,Guam,3.00,0.00,1.40,0.00,2,1,3,...,1.0,206.0,Guam,GUM,838.33,838.33,0.0,430,1,1
4,Jun 08 2022 - 4:30am,Philippines,Yemen,0.00,0.00,1.33,0.33,0,0,0,...,-1.0,155.0,Yemen,YEM,1023.52,1023.52,2.0,191,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280,Mar 29 2022 - 11:30pm,Bolivia,Brazil,0.88,2.63,0.83,2.65,0,4,4,...,1.0,1.0,Brazil,BRA,1841.30,1837.56,0.0,262,-1,-1
3281,Mar 29 2022 - 11:30pm,Ecuador,Argentina,1.47,2.38,1.44,2.29,1,1,2,...,0.0,3.0,Argentina,ARG,1773.88,1770.65,0.0,262,1,0
3282,Mar 29 2022 - 11:30pm,Peru,Paraguay,1.24,0.94,1.33,0.89,2,0,2,...,2.0,47.0,Paraguay,PAR,1448.77,1440.13,-3.0,262,1,1
3283,Mar 29 2022 - 11:30pm,Chile,Uruguay,1.12,1.47,1.06,1.56,0,2,2,...,0.0,14.0,Uruguay,URU,1638.71,1640.95,1.0,262,-1,-1


In [51]:
total_inter

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,year
20505,2018-09-07,Qatar,China,Asia,Asia,98,75,1236,1320,1,...,Win,50.0,72.0,78.0,80.0,79.0,70.2,72.3,71.5,2018
20632,2018-10-12,Qatar,Ecuador,Asia,South America,94,58,1247,1378,4,...,Win,50.0,74.0,78.0,80.0,79.0,74.0,75.7,75.5,2018
20889,2018-12-27,Qatar,Algeria,Asia,Africa,93,67,1258,1347,0,...,Lose,50.0,67.0,78.0,80.0,79.0,77.2,80.0,80.8,2018
20899,2018-12-31,Qatar,Iran,Asia,Asia,93,29,1258,1481,1,...,Lose,50.0,72.0,78.0,80.0,79.0,69.0,73.3,71.0,2018
21279,2019-06-23,Qatar,Argentina,Asia,South America,55,11,1396,1582,0,...,Lose,50.0,81.0,78.0,80.0,79.0,82.5,90.3,84.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,2022-06-14,Poland,Belgium,Europe,Europe,26,2,1544,1827,0,...,Lose,87.0,89.0,75.2,84.7,75.8,80.8,85.7,85.5,2022
23911,2022-06-14,Ukraine,Republic of Ireland,Europe,Europe,27,47,1535,1449,1,...,Draw,75.0,75.0,74.8,78.7,80.0,76.5,72.7,73.8,2022
23912,2022-06-14,Bosnia and Herzegovina,Finland,Europe,Europe,59,57,1388,1406,3,...,Win,76.0,83.0,74.2,77.0,78.0,70.0,72.3,73.5,2022
23913,2022-06-14,Romania,Montenegro,Europe,Europe,48,70,1446,1342,0,...,Lose,77.0,65.0,73.5,73.7,75.0,76.2,74.7,68.2,2022


Here we rename total_inter and take only necessary columns of the dataframe. We then split the dataframe up by year from 2018 to 2022 and then concatenate the results. We then redefine our international_matches dataframe as this result which takes all the cleaning work we did to that dataframe above (with Qatar, USMNT, etc.) and gives us the columns we need displayed by year for the total international matches. We then group by away_team and year to create an away dataset for countries and we group by home_team and year to create a home dataset for countries. We rename teams as needed to prepare for concatenation. We then concatenate the home and away datasets and group by team and year to create our combined dataset. Lastly, we merge full_data with our combined dataset on team1 and team2 to account for when teams play home and away games. This gives us a master dataset that will undergo a bit more cleaning that will then be used for modeling:

In [52]:
international = total_inter
international = international[['date', 'home_team', 'away_team', 'home_team_goalkeeper_score', 
               'away_team_goalkeeper_score', 'home_team_mean_defense_score',
               'home_team_mean_offense_score', 'home_team_mean_midfield_score',
               'away_team_mean_defense_score', 'away_team_mean_offense_score', 
               'away_team_mean_midfield_score', 'year']]

international_2018 = international[international['year']==2018]
international_2019 = international[international['year']==2019]
international_2020 = international[international['year']==2020]
international_2021 = international[international['year']==2021]
international_2022 = international[international['year']==2022]

international_matches = pd.concat([international_2018, international_2019, international_2020,
                       international_2021, international_2022], axis=0).reset_index()

international_matches['year'] = international_matches['year'].apply(str)
international_matches_away = international_matches.groupby(by=['away_team','year']).mean()
international_matches_home = international_matches.groupby(by=['home_team','year']).mean()

home = international_matches_home[international_matches_home.columns.drop(list(international_matches_home.filter(regex='away')))]
home = home.reset_index()
away = international_matches_away[international_matches_away.columns.drop(list(international_matches_away.filter(regex='home')))]
away = away.reset_index()
away = converting_home_and_away(away)
home = home.rename(columns={'home_team' : 'team2', 'home_team_goalkeeper_score': 'team2_goalkeeper_score',
                           'home_team_mean_defense_score' : 'team2_mean_defense_score', 
                           'home_team_mean_offense_score' : 'team2_mean_offense_score',
                           'home_team_mean_midfield_score' : 'team2_mean_midfield_score'})

combined = pd.concat([home,away]).groupby(['team2', 'year']).mean()
combined = combined.reset_index()
full_data['year']= full_data['year'].apply(str)
full_data = full_data.merge(combined, left_on=['team1_name','year'], right_on=['team2','year'], how='left')
full_data = full_data.merge(combined, left_on=['team2_name','year'], right_on=['team2','year'], how='left')
full_data = full_data.dropna()

Imputing values for Qatar and Tunisia:

In [53]:
can = (full_data['team1_name']=='Qatar') & (full_data['team2_name']=='Canada')
full_data.loc[can, ['team1_name', 
                    'team2_name']] = (full_data.loc[can,['team2_name', 'team1_name']].values)
tunisia_2022 = full_data[(full_data['year'] == '2021') & (full_data['team1_name']=='Tunisia')].iloc[0,:].to_frame().T
full_data = tunisia_2022.replace('2021', '2022').append(full_data)
tunisia_2022_1 = full_data[(full_data['year'] == '2020') & (full_data['team2_name']=='Tunisia')].iloc[0,:].to_frame().T
full_data = tunisia_2022_1.replace('2020', '2022').append(full_data)

Renaming columns on our final dataframe after our merge:

In [54]:
full_data = full_data.rename(columns={'team2_goalkeeper_score_x' : 'team1_goalkeeper_score',
                  'team2_mean_defense_score_x' : 'team1_mean_defense_score',
                  'team2_mean_offense_score_x' : 'team1_mean_offense_score',
                  'team2_mean_midfield_score_x' : 'team1_mean_midfield_score',
                  'team2_goalkeeper_score_y' : 'team2_goalkeeper_score',
                  'team2_mean_defense_score_y' : 'team2_mean_defense_score',
                  'team2_mean_offense_score_y' : 'team2_mean_offense_score',
                  'team2_mean_midfield_score_y' : 'team2_mean_midfield_score'})

Dropping necessary columns to prepare for modeling:

In [55]:
model_data = full_data.drop(columns=['team1_goal_timings',
 'team2_goal_timings', 'team1_goal_count',
 'team2_goal_count', 'team2_x',
 'index_x', 'team2_y',
 'index_y', 'date_GMT',
 'year', 'team1_name', 'team2_name', 'first_score_country'])

Dropping necessary columns to prepare for simulation and testing:

In [56]:
full_data = full_data.drop(columns=['team1_goal_timings',
 'team2_goal_timings', 'team1_goal_count',
 'team2_goal_count', 'team2_x',
 'index_x', 'team2_y',
 'index_y', 'first_score_country'])

This is our final cleaned dataset before modeling! Note: we will drop features in the feature selection section below:

In [57]:
model_data

,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,total_goal_count,total_goals_at_half_time,team1_goal_count_half_time,team2_goal_count_half_time,team1_corner_count,team2_corner_count,...,days_since,who_won,team1_goalkeeper_score,team1_mean_defense_score,team1_mean_offense_score,team1_mean_midfield_score,team2_goalkeeper_score,team2_mean_defense_score,team2_mean_offense_score,team2_mean_midfield_score
877,0.00,3.00,0.50,2.00,2,2,1,1,4,5,...,794,0,73.000000,73.000000,77.700000,79.200000,58.00,70.80,68.300,74.50
122,1.50,3.00,2.00,1.50,1,1,1,0,2,8,...,375,1,58.000000,70.800000,68.300000,74.500000,70.00,63.00,64.300,67.00
46,0.00,0.00,2.00,1.33,6,0,0,0,2,5,...,520,0,50.000000,78.000000,80.000000,79.000000,75.00,69.80,70.700,67.50
56,3.00,3.00,3.00,1.80,1,1,1,0,1,3,...,516,1,76.467391,75.640217,77.167391,75.480435,75.75,69.70,72.425,77.05
60,3.00,3.00,2.25,1.50,1,0,0,0,3,6,...,514,1,87.450000,71.225000,69.420000,68.815000,75.70,71.71,76.110,69.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,0.88,2.63,0.83,2.65,4,2,0,2,6,2,...,262,-1,70.000000,68.800000,68.300000,68.800000,89.00,84.80,86.300,85.50
3053,1.47,2.38,1.44,2.29,2,1,0,1,6,2,...,262,0,71.000000,73.500000,76.000000,74.500000,84.00,82.20,89.000,84.00
3054,1.24,0.94,1.33,0.89,2,2,2,0,2,2,...,262,1,74.000000,74.500000,73.000000,76.800000,72.00,74.20,74.300,76.20
3055,1.12,1.47,1.06,1.56,2,0,0,0,5,10,...,262,-1,79.000000,75.500000,76.700000,78.200000,80.00,81.20,84.300,80.00


# Feature Selection:

Function for VIF:

In [58]:
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

In [59]:
model_columns = list(model_data.columns)

In [60]:
# vif_columns.remove('date_GMT')
# vif_columns.remove('team1_name')
# vif_columns.remove('team2_name')
# vif_columns.remove('team1_goal_timings')
# vif_columns.remove('team2_goal_timings')
# vif_columns.remove('team2_goal_count')
# vif_columns.remove('team1_goal_count')
# vif_columns.remove('total_goal_count')
# vif_columns.remove('who_won')
# vif_columns.remove('team1_result_binary')
# vif_columns.remove('team1_result')
# vif_columns.remove('country_abrv_team2')
# vif_columns.remove('country_full_team2')
# vif_columns.remove('country_abrv_team1')
# vif_columns.remove('country_full_team1')
# vif_columns.remove('previous_points_team2')
# vif_columns.remove('previous_points_team1')
# vif_columns.remove('total_points_team2')
# vif_columns.remove('team_a_xg')
# vif_columns.remove('total_points_team1')
# #vif_columns.remove('team1_possession')
# #vif_columns.remove('team2_possession')
# vif_columns.remove('average_goals_per_match_pre_match')
# vif_columns.remove('team_b_xg')
# vif_columns.remove('odds_ft_over35')
# vif_columns.remove('year')
# vif_columns.remove('index_y')
# vif_columns.remove('away_team_y')
# vif_columns.remove('index_x')
# vif_columns.remove('away_team_x')
# vif_columns.remove('over_15_percentage_pre_match')
# vif_columns.remove('odds_ft_team2_win')

After much testing, these are the best features to use to keep VIF values low:

In [61]:
vif_columns = ['Pre_Match_PPG_Home','Pre_Match_PPG_Away',
 'team1_ppg',
 'team2_ppg',
 'team_a_xg',
 'team_b_xg',
 'rank_team1',
 'rank_team2',
 'team1_goalkeeper_score',
 'team1_mean_defense_score',
 'team1_mean_offense_score',
 'team1_mean_midfield_score',
 'team2_goalkeeper_score',
 'team2_mean_defense_score',
 'team2_mean_offense_score',
 'team2_mean_midfield_score',
 'who_won']

Here is our VIF output! Most values are below 5, and we decided to leave those that were higher than 5, but lower than 6.

In [62]:
vif_output = VIF(full_data, vif_columns).to_frame()
vif_output.replace([np.inf, -np.inf], np.nan, inplace=True)
vif_output.dropna(inplace=True)
vif_output.index.values.tolist()
vif_output[vif_output[0]>5].sort_values(by=0, ascending=False)
vif_output

,0
Pre_Match_PPG_Home,1.519403
Pre_Match_PPG_Away,1.900348
team1_ppg,2.116386
team2_ppg,2.486337
team_a_xg,1.372535
team_b_xg,1.323756
rank_team1,3.041567
rank_team2,3.323342
team1_goalkeeper_score,1.694402
team1_mean_defense_score,5.899792


# Splitting the Data for Modeling:

Using the columns from VIF:

In [63]:
final_data = model_data[vif_columns]

In [64]:
from sklearn.model_selection import train_test_split
temp_full_data = final_data
full_data_train, full_data_test = train_test_split(temp_full_data, test_size=0.25, random_state=44)

In [65]:
y_train = full_data_train['who_won']
X_train = full_data_train.drop(['who_won'], axis=1)

y_test = full_data_test['who_won']
X_test = full_data_test.drop(['who_won'], axis=1)

# Models:

## Baseline Model:

Instances of winning appear the most as shown below, so we will test our baseline accuracy by predicting a win for each match:

In [66]:
y_train.value_counts()

 1    414
-1    249
 0    239
Name: who_won, dtype: int64

In [67]:
acc_base = y_train.value_counts()[1] / len(y_train)
print('Baseline Accuracy: ', acc_base)

Baseline Accuracy:  0.458980044345898


## Logistic Regression:

Here we use multinomial logistic regression from statsmodels to output our p-values. Note, there are many features that have high p-values. We have tried dropping these features and using the other features from model_data that are used in the alternative non-working model. However, after much testing with these features, our prediction probabilities are much less skewed and make much more sense than with the features we used in the alternative non-working model. All of our model accuracies will prove to be much better than baseline as well, so we believe our model will make solid predictions. Our team wants to highlight that feature selection was a lot of trial and error due to the amount of data cleaning needed. We see these running models as successes, but also want to highlight our failures (the non-working model). 

In [68]:
#Statsmodels Logistic Regression Model:

log_reg_R = sm.MNLogit(y_train, X_train).fit()
y_pred = log_reg_R.predict(X_test)
print(log_reg_R.summary())

Optimization terminated successfully.
         Current function value: 0.703480
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:                who_won   No. Observations:                  902
Model:                        MNLogit   Df Residuals:                      870
Method:                           MLE   Df Model:                           30
Date:                Fri, 16 Dec 2022   Pseudo R-squ.:                  0.3393
Time:                        21:46:30   Log-Likelihood:                -634.54
converged:                       True   LL-Null:                       -960.33
Covariance Type:            nonrobust   LLR p-value:                5.880e-118
                who_won=0       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Pre_Match_PPG_Home           -0.4177      0.142     -2.946      0.003      -0.

In [69]:
#Sklearn Logistic Regression Model:

clf = LogisticRegression(max_iter=50, multi_class='multinomial')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc_logreg = clf.score(X_test, y_test)
cm_log = confusion_matrix(y_test, y_pred)
y_prob_logreg = clf.predict_proba(X_test)
print('Logistic Regression Test Accuracy:', acc_logreg.round(5))
print(cm_log)

Logistic Regression Test Accuracy: 0.55814
[[ 38   2  32]
 [ 21   6  57]
 [ 15   6 124]]


## SVM:

In [70]:
svm = svm.SVC(kernel = 'linear', C = 1, probability=True, random_state=88).fit(X_train, y_train)
svm_predictions = svm.predict(X_test)
svm_prob = svm.predict_proba(X_test)
acc_svm = svm.score(X_test, y_test)
cm_svm = confusion_matrix(y_test, svm_predictions)
print('SVM Test Accuracy:', acc_svm.round(5))
print(cm_svm)

SVM Test Accuracy: 0.63455
[[ 48  18   6]
 [ 16  26  42]
 [  9  19 117]]


## LDA:

In [71]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_prob_lda = lda.predict_proba(X_test)
lda_prediction = convert_to_result(y_prob_lda)
acc_lda = accuracy_score(y_test, lda_prediction)
cm_lda = confusion_matrix(y_test, lda_prediction)
print('LDA Test Accuracy:', acc_lda.round(5))
print(cm_lda)

LDA Test Accuracy: 0.61462
[[ 46  19   7]
 [ 17  24  43]
 [ 11  19 115]]


## Decision Tree Classifier:

In [72]:
dtc = DecisionTreeClassifier(criterion='entropy',
                             min_samples_leaf=5, 
                             ccp_alpha=0.001,
                             random_state = 88,
                             max_depth=3)

dtc = dtc.fit(X_train, y_train)
y_prob_dtc = dtc.predict_proba(X_test)

In [73]:
dtc_prediction = convert_to_result(y_prob_dtc)
acc_dtc = accuracy_score(y_test, dtc_prediction)
cm_dtc = confusion_matrix(y_test, dtc_prediction)
print("Decision Tree Classifier Accuracy:", acc_dtc.round(5))
print(cm_dtc)

Decision Tree Classifier Accuracy: 0.58472
[[ 51   2  19]
 [ 33   9  42]
 [ 24   5 116]]


## Naive Bayes:

In [74]:
gnb = GaussianNB()
y_prob_bayes_final = gnb.fit(X_train, y_train).predict_proba(X_test)

In [75]:
bayes_final_prediction = convert_to_result(y_prob_bayes_final)
acc_bayes = accuracy_score(y_test, bayes_final_prediction)
cm_bayes = confusion_matrix(y_test, bayes_final_prediction)
print("Naive Bayes Accuracy: ", acc_bayes.round(5))
print(cm_bayes)

Naive Bayes Accuracy:  0.53156
[[ 43  15  14]
 [ 27  17  40]
 [ 19  26 100]]


## Random Forest:

In [76]:
rf = RandomForestClassifier(max_features=4, min_samples_leaf=5, 
                           n_estimators = 500, random_state=88, verbose=0)

rf.fit(X_train, y_train)
rf.verbose = False
y_pred_rf = rf.predict(X_test)

acc_rf = accuracy_score(y_test,y_pred)
cm_rf = confusion_matrix(y_test, y_pred_rf)
print("Random Forest Classifier Accuracy: ", acc_rf.round(5))
print(cm_rf)

Random Forest Classifier Accuracy:  0.55814
[[ 49   5  18]
 [ 20  18  46]
 [ 16   9 120]]


In [77]:
pd.DataFrame({'Feature' : X_train.columns, 
              'Importance score': 100*rf.feature_importances_}).round(1)

,Feature,Importance score
0,Pre_Match_PPG_Home,4.0
1,Pre_Match_PPG_Away,4.1
2,team1_ppg,19.5
3,team2_ppg,16.1
4,team_a_xg,6.4
5,team_b_xg,4.6
6,rank_team1,4.8
7,rank_team2,5.6
8,team1_goalkeeper_score,4.1
9,team1_mean_defense_score,4.3


## Neural Network:

In [78]:
neural_net = MLPClassifier(solver='sgd', alpha=1e-5,
                    hidden_layer_sizes=(100), random_state=88)
neural_net.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=100, random_state=88,
              solver='sgd')

In [79]:
nn_preds = neural_net.predict(X_test)
nn_probs = neural_net.predict_proba(X_test)
acc_nn = neural_net.score(X_test, y_test)
cm_nn = confusion_matrix(y_test, nn_preds)
print('Neural Network Accuracy:', acc_nn.round(5))
print(cm_nn)

Neural Network Accuracy: 0.52824
[[ 42   1  29]
 [ 26   0  58]
 [ 24   4 117]]


End of modeling:

# Model Accuracy Table:

We can see the breakdown of model accuracies here. SVM boasts the highest accuracy, so we will focus our simulations on that model to start:

In [80]:
models = pd.DataFrame({
    'Model': ['Baseline', 'Support Vector Machines', 'Logistic Regression', 
              'LDA', 'Naive Bayes','Decision Tree', 'Random Forest', 'Neural Network'],
    'Accuracy': [acc_base, acc_svm, acc_logreg, 
              acc_lda, acc_bayes, acc_dtc, acc_rf, acc_nn]})
models.round(5).sort_values(by='Accuracy', ascending=False)

,Model,Accuracy
1,Support Vector Machines,0.63455
3,LDA,0.61462
5,Decision Tree,0.58472
2,Logistic Regression,0.55814
6,Random Forest,0.55814
4,Naive Bayes,0.53156
7,Neural Network,0.52824
0,Baseline,0.45898


# Aggregating Data by Country for Simulation:

In [81]:
full_data

,date_GMT,team1_name,team2_name,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,total_goal_count,total_goals_at_half_time,team1_goal_count_half_time,...,days_since,who_won,team1_goalkeeper_score,team1_mean_defense_score,team1_mean_offense_score,team1_mean_midfield_score,team2_goalkeeper_score,team2_mean_defense_score,team2_mean_offense_score,team2_mean_midfield_score
877,Oct 13 2020 - 6:30pm,Nigeria,Tunisia,0.00,3.00,0.50,2.00,2,2,1,...,794,0,73.000000,73.000000,77.700000,79.200000,58.00,70.80,68.300,74.50
122,Dec 06 2021 - 3:00pm,Tunisia,UAE,1.50,3.00,2.00,1.50,1,1,1,...,375,1,58.000000,70.800000,68.300000,74.500000,70.00,63.00,64.300,67.00
46,Jul 14 2021 - 12:50am,Qatar,Panama,0.00,0.00,2.00,1.33,6,0,0,...,520,0,50.000000,78.000000,80.000000,79.000000,75.00,69.80,70.700,67.50
56,Jul 18 2021 - 9:00pm,USMNT,Canada,3.00,3.00,3.00,1.80,1,1,1,...,516,1,76.467391,75.640217,77.167391,75.480435,75.75,69.70,72.425,77.05
60,Jul 20 2021 - 11:00pm,Costa Rica,Jamaica,3.00,3.00,2.25,1.50,1,0,0,...,514,1,87.450000,71.225000,69.420000,68.815000,75.70,71.71,76.110,69.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,Mar 29 2022 - 11:30pm,Bolivia,Brazil,0.88,2.63,0.83,2.65,4,2,0,...,262,-1,70.000000,68.800000,68.300000,68.800000,89.00,84.80,86.300,85.50
3053,Mar 29 2022 - 11:30pm,Ecuador,Argentina,1.47,2.38,1.44,2.29,2,1,0,...,262,0,71.000000,73.500000,76.000000,74.500000,84.00,82.20,89.000,84.00
3054,Mar 29 2022 - 11:30pm,Peru,Paraguay,1.24,0.94,1.33,0.89,2,2,2,...,262,1,74.000000,74.500000,73.000000,76.800000,72.00,74.20,74.300,76.20
3055,Mar 29 2022 - 11:30pm,Chile,Uruguay,1.12,1.47,1.06,1.56,2,0,0,...,262,-1,79.000000,75.500000,76.700000,78.200000,80.00,81.20,84.300,80.00


In [82]:
temp_agg_cols = ['Pre_Match_PPG_Home',
 'Pre_Match_PPG_Away',
 'team1_ppg',
 'team2_ppg',
 'team_a_xg',
 'team_b_xg',
 'team1_name',
 'team2_name',]

Here we select all in-game stats to be aggregated:

In [83]:
in_game_total = full_data[temp_agg_cols]
in_game_total

,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,team_a_xg,team_b_xg,team1_name,team2_name
877,0.00,3.00,0.50,2.00,1.17,1.34,Nigeria,Tunisia
122,1.50,3.00,2.00,1.50,1.72,0.76,Tunisia,UAE
46,0.00,0.00,2.00,1.33,1.59,1.53,Qatar,Panama
56,3.00,3.00,3.00,1.80,0.52,1.19,USMNT,Canada
60,3.00,3.00,2.25,1.50,0.79,1.73,Costa Rica,Jamaica
...,...,...,...,...,...,...,...,...
3052,0.88,2.63,0.83,2.65,1.91,1.90,Bolivia,Brazil
3053,1.47,2.38,1.44,2.29,1.46,0.77,Ecuador,Argentina
3054,1.24,0.94,1.33,0.89,1.32,1.20,Peru,Paraguay
3055,1.12,1.47,1.06,1.56,1.57,2.06,Chile,Uruguay


Here we take the mean of each country's in-game stats:

In [84]:
in_game_agg = full_data[temp_agg_cols].groupby('team1_name').agg(np.average)
in_game_agg.drop(columns=['Pre_Match_PPG_Away', 'team2_ppg', 'team_b_xg'])
in_game_agg

,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,team_a_xg,team_b_xg
team1_name,,,,,,
Albania,0.829286,1.804286,1.282143,1.757143,1.314286,1.405000
Algeria,1.973333,0.974444,2.271111,1.241111,1.533333,1.103333
Argentina,1.729600,1.306800,2.185200,1.436000,1.757200,1.025600
Australia,1.773750,1.183750,1.870000,1.442500,1.785000,1.310000
Austria,1.511905,1.168095,1.796667,1.461905,1.679048,1.179524
...,...,...,...,...,...,...
USMNT,1.948462,1.370000,1.698077,1.521154,1.471154,1.213462
Ukraine,1.678000,1.262000,1.770667,1.426667,1.690000,1.476000
Uruguay,1.741667,1.519444,1.810000,1.371667,1.585000,1.035556


Here we take the positional rankings of each country for the year of 2022 to most accurately represent how the team is rated before the world cup:

In [85]:
temp_agg1 = full_data[['year', 'team1_name', 'rank_team1',
 'team1_goalkeeper_score', 'team1_mean_defense_score',
       'team1_mean_offense_score', 'team1_mean_midfield_score']]
temp_agg2 = temp_agg1[temp_agg1['year'] == '2022']
temp_agg2 = temp_agg2.groupby('team1_name').agg(np.mean)
temp_agg2

,rank_team1,team1_goalkeeper_score,team1_mean_defense_score,team1_mean_offense_score,team1_mean_midfield_score
team1_name,,,,,
Albania,66.0,80.0,76.2,70.0,73.0
Algeria,37.0,78.0,78.0,81.0,78.0
Argentina,3.0,84.0,82.2,89.0,84.0
Australia,38.0,77.0,72.0,72.3,73.5
Austria,34.0,74.0,80.2,77.0,80.8
...,...,...,...,...,...
USMNT,16.0,77.0,75.8,77.7,75.8
Ukraine,27.0,75.0,74.8,78.7,80.0
Uruguay,14.0,80.0,81.2,84.3,80.0


In [86]:
first_score_agg = full_data[['year', 'team1_name']]
first_score_agg = first_score_agg[first_score_agg['year'] == '2022']
first_score_agg = first_score_agg.groupby('team1_name').agg(lambda x: pd.Series.mode(x)[0])
first_score_agg

,year
team1_name,
Albania,2022
Algeria,2022
Argentina,2022
Australia,2022
Austria,2022
...,...
USMNT,2022
Ukraine,2022
Uruguay,2022


Here we merge all three dataframes above to get a master aggregate dataframe that has all of the averages we need for each team:

In [87]:
merged_agg = in_game_agg.merge(temp_agg2, how = 'left', on= 'team1_name')
merged_agg2 = merged_agg.merge(first_score_agg,  how = 'left', on= 'team1_name')
merged_agg2 = merged_agg2.drop(columns=['Pre_Match_PPG_Away',
                                      'team2_ppg', 'team_b_xg' ]).reset_index()
merged_agg2

,team1_name,Pre_Match_PPG_Home,team1_ppg,team_a_xg,rank_team1,team1_goalkeeper_score,team1_mean_defense_score,team1_mean_offense_score,team1_mean_midfield_score,year
0,Albania,0.829286,1.282143,1.314286,66.0,80.0,76.2,70.0,73.0,2022
1,Algeria,1.973333,2.271111,1.533333,37.0,78.0,78.0,81.0,78.0,2022
2,Argentina,1.729600,2.185200,1.757200,3.0,84.0,82.2,89.0,84.0,2022
3,Australia,1.773750,1.870000,1.785000,38.0,77.0,72.0,72.3,73.5,2022
4,Austria,1.511905,1.796667,1.679048,34.0,74.0,80.2,77.0,80.8,2022
...,...,...,...,...,...,...,...,...,...,...
73,USMNT,1.948462,1.698077,1.471154,16.0,77.0,75.8,77.7,75.8,2022
74,Ukraine,1.678000,1.770667,1.690000,27.0,75.0,74.8,78.7,80.0,2022
75,Uruguay,1.741667,1.810000,1.585000,14.0,80.0,81.2,84.3,80.0,2022
76,Venezuela,0.922500,0.952500,1.391875,57.0,73.0,72.8,76.3,77.8,2022


# Testing Models on Specific Countries:

This function pulls all of the features for two specific countries that we enter as the parameters and merges the data into a one-row dataframe with one half of the data representing one country and the other half of the data representing the other country. With this function, we can see the outcome between any two countries of the world cup. We will test our function below, but for simulation we will only simulate combinations that actually happen in the group stage of the world cup:

In [88]:
#Function that pulls all features for two specific countries that will play in a match:
def getting_data_from_team2(country_name1, country_name2):
    ### Features feeded into the model:
    ## When using X_train, values are all 0. When using X_train_final, values are positive numbers.
    ## We want to use X_train, need to figure out what is wrong:
    temp_dict = {}
    for feature in list(merged_agg2.columns)[1:]:
        temp_dict[feature] = [list(merged_agg2[merged_agg2['team1_name'] == country_name1][feature])[0]]
    for feature in list(merged_agg2.columns)[1:]:
        temp_dict[feature+'2'] = [list(merged_agg2[merged_agg2['team1_name'] == country_name2][feature])[0]]
    final_df = pd.DataFrame(temp_dict).rename(columns={'Pre_Match_PPG_Home2': 'Pre_Match_PPG_Away',
                                               'team1_ppg2' : 'team2_ppg',
                                               'team_a_xg2' : 'team_b_xg',
                                               'rank_team12': 'rank_team2',
                                               'team1_goalkeeper_score2' : 'team2_goalkeeper_score',
                                               'team1_mean_defense_score2' : 'team2_mean_defense_score',
                                               'team1_mean_offense_score2' : 'team2_mean_offense_score',
                                               'team1_mean_midfield_score2' : 'team2_mean_midfield_score',})
    final_df = final_df[['Pre_Match_PPG_Home', 'Pre_Match_PPG_Away', 'team1_ppg', 'team2_ppg',
       'team_a_xg', 'team_b_xg', 'rank_team1', 'rank_team2',
       'team1_goalkeeper_score', 'team1_mean_defense_score',
       'team1_mean_offense_score', 'team1_mean_midfield_score',
       'team2_goalkeeper_score', 'team2_mean_defense_score',
       'team2_mean_offense_score', 'team2_mean_midfield_score']]
                                               
    return final_df

Testing the function between two random countries:

In [89]:
getting_data_from_team2('France', 'England')

,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,team_a_xg,team_b_xg,rank_team1,rank_team2,team1_goalkeeper_score,team1_mean_defense_score,team1_mean_offense_score,team1_mean_midfield_score,team2_goalkeeper_score,team2_mean_defense_score,team2_mean_offense_score,team2_mean_midfield_score
0,1.719,1.798846,2.226333,2.202692,1.953,1.768462,4.0,5.0,87.0,84.2,88.3,86.8,83.0,85.0,88.0,84.0


For all models: The first item indicates the probability of the second country (right input) winning, the second item indicates the probability of a draw, and the third item indicates the probability of first country (left input) winning.

## LDA test:

For the test below: Brazil has a 64.7% chance of winning, Costa Rica has a 3.3% chance of winning, and there is a 32% chance of a draw.

In [92]:
lda.predict_proba(getting_data_from_team2('Brazil', 'Costa Rica'))

array([[0.03304936, 0.31984184, 0.6471088 ]])

In [93]:
lda.predict_proba(getting_data_from_team2('England', 'France'))

array([[0.38455143, 0.31257764, 0.30287094]])

## Logistic Regression test:

In [96]:
clf.predict_proba(getting_data_from_team2('Spain', 'Argentina'))

array([[0.30115466, 0.358531  , 0.34031433]])

In [97]:
clf.predict_proba(getting_data_from_team2('England', 'France'))

array([[0.336169  , 0.26451261, 0.39931839]])

## Decision Tree Classifier test:

In [98]:
dtc.predict_proba(getting_data_from_team2('England', 'France'))

array([[0.52702703, 0.18918919, 0.28378378]])

In [99]:
dtc.predict_proba(getting_data_from_team2('Iran', 'USMNT'))

array([[0.15929204, 0.36283186, 0.47787611]])

## Naive Bayes Test

In [100]:
gnb.predict_proba(getting_data_from_team2('Iran', 'France'))

array([[9.90887143e-01, 9.05265775e-03, 6.01988623e-05]])

In [101]:
gnb.predict_proba(getting_data_from_team2('Tunisia', 'Denmark'))

array([[0.94467966, 0.0540094 , 0.00131094]])

## Amy's Model test:

In [102]:
rf.predict_proba(getting_data_from_team2('South Korea', 'Netherlands'))

array([[0.34853206, 0.37049244, 0.28097549]])

In [104]:
rf.predict_proba(getting_data_from_team2('Tunisia', 'Denmark'))

array([[0.49778183, 0.30969528, 0.19252289]])

## Neural Net Test:

In [103]:
neural_net.predict_proba(getting_data_from_team2('Portugal', 'Uruguay'))

array([[0.20206385, 0.31843007, 0.47950608]])

In [105]:
neural_net.predict_proba(getting_data_from_team2('Ghana', 'Cameroon'))

array([[0.24541558, 0.35085818, 0.40372624]])

# Simulation Function:

In [106]:
qatar_group_stage = qatar_group_stage.replace(['IR Iran'], 'Iran')
qatar_group_stage = qatar_group_stage.replace(['USA'], 'USMNT')
qatar_group_stage = qatar_group_stage.replace(['Korea Republic'], 'South Korea')
gs = qatar_group_stage.iloc[0:48, :]
gs

,country1,country2,group
0,Qatar,Ecuador,a
1,Senegal,Netherlands,a
2,England,Iran,b
3,USMNT,Wales,b
4,France,Australia,d
5,Denmark,Tunisia,d
6,Mexico,Poland,c
7,Argentina,Saudi Arabia,c
8,Belgium,Canada,f
9,Spain,Costa Rica,e


This function assigns point values to each team depending on if they win, lose, or draw. This function also defines what makes a win, loss, or draw. There is an added threshold for draws to account for predictions that are very close, which will be classified as draws in the simulation. Each team plays three times, and can get three points for winning a game, one point for drawing a game, and zero points for losing a game: Note: THIS IS THE FUNCTION WE ARE USING

In [103]:
# Function to simulate group stage:
def group_stage_predictions2(team1, team2, model):

    team1_points = 0
    team2_points = 0

    team1_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(2)
    team2_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(0)
    draw_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(1)

    num_points = 0
    if draw_prob > team1_win_prob-0.075 and draw_prob > team2_win_prob-.075:
        #print('Draw')
        team2_points += 1
        team1_points += 1
    elif team2_win_prob > team1_win_prob and team2_win_prob > draw_prob:
        #print(team2 + "wins with " + team2_win_prob + "probability")
        team2_points += 3
    elif  team1_win_prob > team2_win_prob and team1_win_prob > draw_prob:
        #print(team1 + "wins with " + team1_win_prob + "probability")
        team1_points += 3
    return([team1_points, team2_points])


In [104]:
wc_group_matches = gs[['country1', 'country2', 'group']].sort_values('group')
print(wc_group_matches)

        country1      country2 group
0          Qatar       Ecuador     a
1        Senegal   Netherlands     a
35   Netherlands         Qatar     a
34       Ecuador       Senegal     a
18   Netherlands       Ecuador     a
17         Qatar       Senegal     a
2        England          Iran     b
3          USMNT         Wales     b
33          Iran         USMNT     b
32         Wales       England     b
19       England         USMNT     b
16         Wales          Iran     b
39  Saudi Arabia        Mexico     c
38        Poland     Argentina     c
21        Poland  Saudi Arabia     c
23     Argentina        Mexico     c
6         Mexico        Poland     c
7      Argentina  Saudi Arabia     c
36     Australia       Denmark     d
37       Tunisia        France     d
20       Tunisia     Australia     d
22        France       Denmark     d
4         France     Australia     d
5        Denmark       Tunisia     d
9          Spain    Costa Rica     e
42         Japan         Spain     e
2

This function also adds point values depending on the outcome of the game. Instead of using the concrete probability values, this function uses those probabilities as weights and makes a random choice. This was in attempt to add some noise to our model. However, through testing we found that this function was too variable for what we were trying to accomplish, so we decided to use the first function, not this one. Note: WE ARE NOT USING THIS FUNCTION.

In [106]:
groups = qatar_group_stage['group'].unique()

# Function to simulate group stage:
def group_stage_predictions(team1, team2, model):

    team1_points = 0
    team2_points = 0

    team1_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(2)
    team2_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(0)
    draw_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(1)

    result = random.choices([team1, 'Draw', team2], weights = [team2_win_prob, draw_prob, team1_win_prob], k = 100)
    counter = Counter(result)
    most_common = counter.most_common(1)[0][0]
    if most_common == 'Draw':
        team1_points +=1
        team2_points +=1
        print('Draw')
    if most_common == team1:
        team1_points +=3
        print(team1 + " wins!")
    if most_common == team2:
        team2_points +=3
        print(team2 + " wins!")
    return [team1_points, team2_points]


In [107]:
wc_group_matches = wc_group_matches.reset_index()

# Group Stage Tables:

Simulating the whole group stage and creating and ordered dataframe by points of the results:

In [108]:

def simulation(group_stage_df, model):
    
    teams = []
    r = []
    groups = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    group = ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'c',
       'c', 'c', 'c', 'd', 'd', 'd', 'd', 'e', 'e',
       'e', 'e', 'f', 'f', 'f', 'f', 'g', 'g', 'g',
       'g', 'h', 'h', 'h', 'h']
    
    for g in groups:
        t = list(group_stage_df[(group_stage_df['group']==g)]['country1'].unique())
        teams.append(t)
    group_stage_df['Points'] = gs.apply(lambda x: group_stage_predictions2(x.country1,
                                                                           x.country2, model), axis=1)
    
    for team in teams:
        for i in np.arange(4):
            country = team[i]
            c = 0
            for i in group_stage_df[(group_stage_df['country1']==country)]['Points']:
                c += i[0]
            for j in group_stage_df[(group_stage_df['country2']==country)]['Points']:
                c += j[1]
            r.append(c)
    expected_score_df = pd.DataFrame(list(zip([item for sublist in teams for item in sublist],
                                              r)),columns =['Country', 'Expected Score'])
    expected_score_df['Group'] = group
    expected_score_df = expected_score_df.sort_values('Expected Score', ascending=False)
    
    return expected_score_df
    

In [109]:
expected_score_df = simulation(gs, svm)
expected_score_df

,Country,Expected Score,Group
16,Spain,9,e
29,Portugal,9,h
12,France,9,d
25,Brazil,9,g
20,Belgium,9,f
9,Argentina,9,c
4,England,9,b
2,Netherlands,7,a
7,Iran,4,b
27,Serbia,4,g


Each group outcome is shown below with an expected score value:

In [110]:
group_a = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'a').dropna()
group_a

,Country,Expected Score,Group
2,Netherlands,7.0,a
1,Senegal,3.0,a
0,Qatar,2.0,a
3,Ecuador,2.0,a


In [111]:
group_b = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'b').dropna()
group_b

,Country,Expected Score,Group
4,England,9.0,b
7,Iran,4.0,b
6,Wales,2.0,b
5,USMNT,1.0,b


In [112]:
group_c = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'c').dropna()
group_c

,Country,Expected Score,Group
9,Argentina,9.0,c
10,Poland,4.0,c
8,Mexico,2.0,c
11,Saudi Arabia,1.0,c


In [113]:
group_d = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'd').dropna()
group_d

,Country,Expected Score,Group
12,France,9.0,d
13,Denmark,4.0,d
15,Australia,2.0,d
14,Tunisia,1.0,d


In [114]:
group_e = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'e').dropna()
group_e

,Country,Expected Score,Group
16,Spain,9.0,e
18,Japan,4.0,e
17,Germany,2.0,e
19,Costa Rica,1.0,e


In [115]:
group_f = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'f').dropna()
group_f

,Country,Expected Score,Group
20,Belgium,9.0,f
22,Croatia,4.0,f
23,Canada,3.0,f
21,Morocco,1.0,f


In [116]:
group_g = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'g').dropna()
group_g

,Country,Expected Score,Group
25,Brazil,9.0,g
27,Serbia,4.0,g
24,Switzerland,4.0,g
26,Cameroon,0.0,g


In [117]:
group_h = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'h').dropna()
group_h

,Country,Expected Score,Group
29,Portugal,9.0,h
30,South Korea,4.0,h
28,Uruguay,2.0,h
31,Ghana,1.0,h


## Knockout Stages: